## Define Basic Field and Term Classes

In [1]:
class field(object):
    def __init__(self, symbol, massDim, lorentz_rank, spinor_rank, spinor_rank_conj):
        self.symbol = symbol #string symbol for field
        self.lorentz_rank = lorentz_rank #int indicating lorentz rank of field
        self.spinor_rank = spinor_rank #int indicating spinor rank of field
        self.spinor_rank_conj = spinor_rank_conj #int indicating spinor rank of field
        self.massDim= massDim #int indicating mass dimension of field
    def info(self):
        return 'symbol: ' + str(self.symbol) \
            + ' massDim: ' + str(self.massDim) \
            + ' lorentz_rank: ' + str(self.lorentz_rank) \
            + ' spinor_rank: ' + str(self.spinor_rank)
    def get_symbol(self):
        return self.symbol
    def get_lorentz_rank(self):
        return self.lorentz_rank
    def get_spinor_rank(self):
        return self.spinor_rank
    def get_spinor_rank_conj(self):
        return self.spinor_rank_conj
    def get_massDim(self):
        return self.massDim
    def __eq__(self, other):
        eq = (self.symbol == other.symbol)
        return eq

## Generate All Multisets of Derivatives and Fields of a Given Mass Dimension

In [2]:

def totalMassDim(field_multiplicities, field_massDims):
    if len(field_multiplicities) != len(field_massDims):
        print('totalMassDim(): Input lists must have same length')
        return None
    n_types = len(field_multiplicities)
    total_massDim = 0
    for i in range(n_types):
        total_massDim += field_multiplicities[i]*field_massDims[i]
    return total_massDim

def totalLorentzRank(field_multiplicities, field_lorentzRanks):
    if len(field_multiplicities) != len(field_massDims):
        print('totalLorentzRank(): Input lists must have same length')
        return None
    n_types = len(field_multiplicities)
    total_lorentzRank = 0
    for i in range(n_types):
        total_lorentzRank += field_multiplicities[i]*field_lorentzRanks[i]
    return total_lorentzRank

def convert_to_dict(field_multiplicities, field_types, remove_zeros):
    # here, field_types = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp'], but may be adjusted to include more
    # field types (for example, in theories with multiple Fermi fields or non-Abelian gauge fields) if necessary.
    if len(field_multiplicities) != len(field_types):
        print('Length of field_multiplicities input to convert_to_dict() must be equal to length of field_types.')
        return None
    n_types = len(field_types)
    field_multiset = {}
    if remove_zeros:
        for i in range(n_types):
            if field_multiplicities[i] != 0:
                field_multiset[field_types[i]] = field_multiplicities[i]
        return field_multiset
    else:
        for i in range(n_types):
            field_multiset[field_types[i]] = field_multiplicities[i]
        return field_multiset
    
def generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks):
    n_types = len(field_symbols)
    zero_list = [0 for i in range(len(field_symbols))]
    front = [zero_list]
    field_multiplicities_dict = {}
    
    counter = 0
    while front:
        #print(counter)
        front_new = front.copy()
        for i in range(len(front)):
            #print('i: ' + str(i))
            multiplicities_list = front[i]
            #print(multiplicities_list)
            # find index last non-zero element
            if multiplicities_list == len(field_symbols)*[0]:
                index_last_nonzero = 0
            else: 
                index_last_nonzero = [i for (i, m) in enumerate(multiplicities_list) if m != 0][-1]
                
            for j in range(index_last_nonzero, n_types):
                multiplicities_list_new = multiplicities_list.copy()
                multiplicities_list_new[j] += 1
                M = totalMassDim(multiplicities_list_new, field_massDims)
                L = totalLorentzRank(multiplicities_list_new, field_lorentzRanks)
                if M < massDim:
                    #print('front_new before append:' + str(front_new))
                    front_new.append(multiplicities_list_new)
                    #print('front_new after append:' + str(front_new))
                if M <= massDim and L%2 == 0:
                    try:
                        field_multiplicities_dict[M].append(multiplicities_list_new)
                    except:
                        field_multiplicities_dict[M] = [multiplicities_list_new]
            #print('len(front): ' + str(len(front)))
            #print('len(front_new): ' + str(len(front_new)))
            #del front_new[i]
            front_new.remove(multiplicities_list)
        front = front_new
        counter += 1
    
    # create field multisets dict, where each value is a dict rather than a list
    field_multisets_dict = {}
    
    for key in field_multiplicities_dict.keys():
        field_multisets_dict[key] = []
        field_multiplicities_list = field_multiplicities_dict[key]
        for field_multiplicities in field_multiplicities_list:
            # if number of derivatives (field_multiplicities[0]) equal to mass dimension (key), do not include
            # since multiset is all derivatives
            if field_multiplicities[0] == key: 
                continue
            field_multiset = convert_to_dict(field_multiplicities, field_symbols, True)
            field_multisets_dict[key].append(field_multiset)   
    return field_multisets_dict


In [3]:
massDim = 6
field_symbols = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp']
field_massDims = [1, 2, 3, 3, 3 ,3 ,3]
field_lorentzRanks = [1, 2, 0, 1, 2, 1, 0]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    #print('len(field_multisets_dict[' + str(key) + ']): ' + str(len(value)))
    

2
[{'F': 1}]
3
[{'S': 1}, {'T': 1}, {'Sp': 1}]
4
[{'D': 1, 'V': 1}, {'D': 1, 'Vp': 1}, {'F': 2}, {'D': 2, 'F': 1}]
5
[{'F': 1, 'S': 1}, {'F': 1, 'T': 1}, {'F': 1, 'Sp': 1}, {'D': 2, 'S': 1}, {'D': 2, 'T': 1}, {'D': 2, 'Sp': 1}]
6
[{'S': 2}, {'S': 1, 'T': 1}, {'S': 1, 'Sp': 1}, {'V': 2}, {'V': 1, 'Vp': 1}, {'T': 2}, {'T': 1, 'Sp': 1}, {'Vp': 2}, {'Sp': 2}, {'D': 1, 'F': 1, 'V': 1}, {'D': 1, 'F': 1, 'Vp': 1}, {'F': 3}, {'D': 3, 'V': 1}, {'D': 3, 'Vp': 1}, {'D': 2, 'F': 2}, {'D': 4, 'F': 1}]


In [4]:
massDim = 6
field_symbols = ['D', 'F', 'S', 'V', 'T', 'Vp', 'Sp']
field_massDims = [1, 2, 3, 3, 3 ,3 ,3]
field_lorentzRanks = [1, 2, 0, 1, 2, 1, 0]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    #print('len(field_multisets_dict[' + str(key) + ']): ' + str(len(value)))

2
[{'F': 1}]
3
[{'S': 1}, {'T': 1}, {'Sp': 1}]
4
[{'D': 1, 'V': 1}, {'D': 1, 'Vp': 1}, {'F': 2}, {'D': 2, 'F': 1}]
5
[{'F': 1, 'S': 1}, {'F': 1, 'T': 1}, {'F': 1, 'Sp': 1}, {'D': 2, 'S': 1}, {'D': 2, 'T': 1}, {'D': 2, 'Sp': 1}]
6
[{'S': 2}, {'S': 1, 'T': 1}, {'S': 1, 'Sp': 1}, {'V': 2}, {'V': 1, 'Vp': 1}, {'T': 2}, {'T': 1, 'Sp': 1}, {'Vp': 2}, {'Sp': 2}, {'D': 1, 'F': 1, 'V': 1}, {'D': 1, 'F': 1, 'Vp': 1}, {'F': 3}, {'D': 3, 'V': 1}, {'D': 3, 'Vp': 1}, {'D': 2, 'F': 2}, {'D': 4, 'F': 1}]


In [5]:
massDim = 4
field_symbols = ['A', 'B', 'C']
field_massDims = [1, 2, 3]
field_lorentzRanks = [1, 2, 1]
field_multisets_dict = generate_field_multisets(massDim, field_symbols, field_massDims, field_lorentzRanks)
for (key, value) in field_multisets_dict.items():
    print(key)
    print(value)
    print('len(field_multisets_dict[m]): ' + str(len(value)))

2
[{'B': 1}]
len(field_multisets_dict[m]): 1
4
[{'A': 1, 'C': 1}, {'B': 2}, {'A': 2, 'B': 1}]
len(field_multisets_dict[m]): 3


## Generate All Possible Derivative Assignments for a Given Multiset of Fields and Derivatives

In [6]:
# tuples_sum() implementation taken from http://code.activestate.com/recipes/578608-generates-tuples-of-integers-with-a-given-sum/
from itertools import permutations

def tuples_sum(nbval,total,order=True) :
    """ 
        Generate all the tuples L of nbval positive or nul integer 
        such that sum(L)=total.
        The tuples may be ordered (decreasing order) or not
    """
    if nbval == 0 and total == 0 : yield tuple() ; return #raise StopIteration
    if nbval == 1 : yield (total,) ; return #raise StopIteration
    if total==0 : yield (0,)*nbval ; return #raise StopIteration
    for start in range(total,0,-1) :
        for qu in tuples_sum(nbval-1,total-start) :
            if qu[0]<=start :
                sol=(start,)+qu
                if order : yield sol
                else :
                    l=set()
                    for p in permutations(sol,len(sol)) :
                        if p not in l :
                            l.add(p)
                            yield p
    
if __name__=='__main__' :
    print("How to obtain 5 by adding a+b+c (>=0) ? ")
    print("Give me the list of (a,b,c) tuples.")
    g=tuples_sum(3,6,order=False)
    print(list(g))
    
    print("How to obtain 6 by adding 3 positive or nul integers ?")
    g=tuples_sum(3,6,order=True)
    print(list(g))

How to obtain 5 by adding a+b+c (>=0) ? 
Give me the list of (a,b,c) tuples.
[(6, 0, 0), (0, 6, 0), (0, 0, 6), (5, 1, 0), (5, 0, 1), (1, 5, 0), (1, 0, 5), (0, 5, 1), (0, 1, 5), (4, 2, 0), (4, 0, 2), (2, 4, 0), (2, 0, 4), (0, 4, 2), (0, 2, 4), (4, 1, 1), (1, 4, 1), (1, 1, 4), (3, 3, 0), (3, 0, 3), (0, 3, 3), (3, 2, 1), (3, 1, 2), (2, 3, 1), (2, 1, 3), (1, 3, 2), (1, 2, 3), (2, 2, 2)]
How to obtain 6 by adding 3 positive or nul integers ?
[(6, 0, 0), (5, 1, 0), (4, 2, 0), (4, 1, 1), (3, 3, 0), (3, 2, 1), (2, 2, 2)]


In [7]:
n = 4
m = 7
list(tuples_sum(m,n,order=False))


[(4, 0, 0, 0, 0, 0, 0),
 (0, 4, 0, 0, 0, 0, 0),
 (0, 0, 4, 0, 0, 0, 0),
 (0, 0, 0, 4, 0, 0, 0),
 (0, 0, 0, 0, 4, 0, 0),
 (0, 0, 0, 0, 0, 4, 0),
 (0, 0, 0, 0, 0, 0, 4),
 (3, 1, 0, 0, 0, 0, 0),
 (3, 0, 1, 0, 0, 0, 0),
 (3, 0, 0, 1, 0, 0, 0),
 (3, 0, 0, 0, 1, 0, 0),
 (3, 0, 0, 0, 0, 1, 0),
 (3, 0, 0, 0, 0, 0, 1),
 (1, 3, 0, 0, 0, 0, 0),
 (1, 0, 3, 0, 0, 0, 0),
 (1, 0, 0, 3, 0, 0, 0),
 (1, 0, 0, 0, 3, 0, 0),
 (1, 0, 0, 0, 0, 3, 0),
 (1, 0, 0, 0, 0, 0, 3),
 (0, 3, 1, 0, 0, 0, 0),
 (0, 3, 0, 1, 0, 0, 0),
 (0, 3, 0, 0, 1, 0, 0),
 (0, 3, 0, 0, 0, 1, 0),
 (0, 3, 0, 0, 0, 0, 1),
 (0, 1, 3, 0, 0, 0, 0),
 (0, 1, 0, 3, 0, 0, 0),
 (0, 1, 0, 0, 3, 0, 0),
 (0, 1, 0, 0, 0, 3, 0),
 (0, 1, 0, 0, 0, 0, 3),
 (0, 0, 3, 1, 0, 0, 0),
 (0, 0, 3, 0, 1, 0, 0),
 (0, 0, 3, 0, 0, 1, 0),
 (0, 0, 3, 0, 0, 0, 1),
 (0, 0, 1, 3, 0, 0, 0),
 (0, 0, 1, 0, 3, 0, 0),
 (0, 0, 1, 0, 0, 3, 0),
 (0, 0, 1, 0, 0, 0, 3),
 (0, 0, 0, 3, 1, 0, 0),
 (0, 0, 0, 3, 0, 1, 0),
 (0, 0, 0, 3, 0, 0, 1),
 (0, 0, 0, 1, 3, 0, 0),
 (0, 0, 0, 1, 0,

In [8]:
m = 2
n = 5
list(tuples_sum(m,n,order=False))
   

[(5, 0), (0, 5), (4, 1), (1, 4), (3, 2), (2, 3)]

In [9]:
#import inspect # this is to check recursion depth for debugging

def generate_pair_partitions(inner_partition, i_start):
    # INPUT:
    # inner_partition: sorted list (non-increasing order) of numbers that sum to some n_Di, an element of the outer
    # partition of n_D. 
    # OUTPUT:
    # pair_partitions_list: a list of lists of tuples, where each tuple has two elements that sum to the corresponding
    # element of the inner_partition. The number of tuples in each sublist matches the number of elements in 
    # inner_partition. 
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to partition elements of inner partition into two
    # non-negative integers (indicating number of derivatives acting on psi_bar and psi, respectively). 

    # base case
    #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
    if len(inner_partition) == 1:
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list = [[item] for item in pair_partitions]
        return pair_partitions_list
    
    # if two successive elements of inner_partition are equal, then different orderings of 
    # corresponding pair partitions [..., (a,b), (c,d), ...] and [..., (c,d), (a,b), ...] are equal. To avoid
    # repeats, need to modify recursion step for case when successive elements are equal. 
    if inner_partition[1] == inner_partition[0]:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))
        for i in range(i_start, len(pair_partitions)):
            pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], i)
            #print('RECURSION DEPTH: ' + str(len(inspect.stack(0))-29))
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    
    # recursion step is simpler in case where successive elements are not equal
    else:
        pair_partitions_list = []
        pair_partitions = list(tuples_sum(2, inner_partition[0], order=False))[i_start:]
        pair_partitions_list_old = generate_pair_partitions(inner_partition[1:], 0)
        for i in range(len(pair_partitions)):
            extension = pair_partitions[i]
            for j in range(len(pair_partitions_list_old)):
                pair_partition_old = pair_partitions_list_old[j]
                pair_partition_new = [extension] + pair_partition_old 
                pair_partitions_list.append(pair_partition_new)
        return pair_partitions_list
    


In [10]:
inner_partition = [3, 2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (2, 0), (2, 0), (1, 0)],
 [(3, 0), (2, 0), (2, 0), (0, 1)],
 [(3, 0), (2, 0), (0, 2), (1, 0)],
 [(3, 0), (2, 0), (0, 2), (0, 1)],
 [(3, 0), (2, 0), (1, 1), (1, 0)],
 [(3, 0), (2, 0), (1, 1), (0, 1)],
 [(3, 0), (0, 2), (0, 2), (1, 0)],
 [(3, 0), (0, 2), (0, 2), (0, 1)],
 [(3, 0), (0, 2), (1, 1), (1, 0)],
 [(3, 0), (0, 2), (1, 1), (0, 1)],
 [(3, 0), (1, 1), (1, 1), (1, 0)],
 [(3, 0), (1, 1), (1, 1), (0, 1)],
 [(0, 3), (2, 0), (2, 0), (1, 0)],
 [(0, 3), (2, 0), (2, 0), (0, 1)],
 [(0, 3), (2, 0), (0, 2), (1, 0)],
 [(0, 3), (2, 0), (0, 2), (0, 1)],
 [(0, 3), (2, 0), (1, 1), (1, 0)],
 [(0, 3), (2, 0), (1, 1), (0, 1)],
 [(0, 3), (0, 2), (0, 2), (1, 0)],
 [(0, 3), (0, 2), (0, 2), (0, 1)],
 [(0, 3), (0, 2), (1, 1), (1, 0)],
 [(0, 3), (0, 2), (1, 1), (0, 1)],
 [(0, 3), (1, 1), (1, 1), (1, 0)],
 [(0, 3), (1, 1), (1, 1), (0, 1)],
 [(2, 1), (2, 0), (2, 0), (1, 0)],
 [(2, 1), (2, 0), (2, 0), (0, 1)],
 [(2, 1), (2, 0), (0, 2), (1, 0)],
 [(2, 1), (2, 0), (0, 2), (0, 1)],
 [(2, 1), (2, 0), (1

In [11]:
inner_partition = [3, 3, 2, 2]
generate_pair_partitions(inner_partition, 0)

[[(3, 0), (3, 0), (2, 0), (2, 0)],
 [(3, 0), (3, 0), (2, 0), (0, 2)],
 [(3, 0), (3, 0), (2, 0), (1, 1)],
 [(3, 0), (3, 0), (0, 2), (0, 2)],
 [(3, 0), (3, 0), (0, 2), (1, 1)],
 [(3, 0), (3, 0), (1, 1), (1, 1)],
 [(3, 0), (0, 3), (2, 0), (2, 0)],
 [(3, 0), (0, 3), (2, 0), (0, 2)],
 [(3, 0), (0, 3), (2, 0), (1, 1)],
 [(3, 0), (0, 3), (0, 2), (0, 2)],
 [(3, 0), (0, 3), (0, 2), (1, 1)],
 [(3, 0), (0, 3), (1, 1), (1, 1)],
 [(3, 0), (2, 1), (2, 0), (2, 0)],
 [(3, 0), (2, 1), (2, 0), (0, 2)],
 [(3, 0), (2, 1), (2, 0), (1, 1)],
 [(3, 0), (2, 1), (0, 2), (0, 2)],
 [(3, 0), (2, 1), (0, 2), (1, 1)],
 [(3, 0), (2, 1), (1, 1), (1, 1)],
 [(3, 0), (1, 2), (2, 0), (2, 0)],
 [(3, 0), (1, 2), (2, 0), (0, 2)],
 [(3, 0), (1, 2), (2, 0), (1, 1)],
 [(3, 0), (1, 2), (0, 2), (0, 2)],
 [(3, 0), (1, 2), (0, 2), (1, 1)],
 [(3, 0), (1, 2), (1, 1), (1, 1)],
 [(0, 3), (0, 3), (2, 0), (2, 0)],
 [(0, 3), (0, 3), (2, 0), (0, 2)],
 [(0, 3), (0, 3), (2, 0), (1, 1)],
 [(0, 3), (0, 3), (0, 2), (0, 2)],
 [(0, 3), (0, 3), (0

In [12]:
inner_partition = [2, 2, 1]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (1, 0)],
 [(2, 0), (2, 0), (0, 1)],
 [(2, 0), (0, 2), (1, 0)],
 [(2, 0), (0, 2), (0, 1)],
 [(2, 0), (1, 1), (1, 0)],
 [(2, 0), (1, 1), (0, 1)],
 [(0, 2), (0, 2), (1, 0)],
 [(0, 2), (0, 2), (0, 1)],
 [(0, 2), (1, 1), (1, 0)],
 [(0, 2), (1, 1), (0, 1)],
 [(1, 1), (1, 1), (1, 0)],
 [(1, 1), (1, 1), (0, 1)]]

In [13]:
inner_partition = [2, 2]
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0)],
 [(2, 0), (0, 2)],
 [(2, 0), (1, 1)],
 [(0, 2), (0, 2)],
 [(0, 2), (1, 1)],
 [(1, 1), (1, 1)]]

In [14]:
inner_partition = (2, 2, 2)
generate_pair_partitions(inner_partition, 0)

[[(2, 0), (2, 0), (2, 0)],
 [(2, 0), (2, 0), (0, 2)],
 [(2, 0), (2, 0), (1, 1)],
 [(2, 0), (0, 2), (0, 2)],
 [(2, 0), (0, 2), (1, 1)],
 [(2, 0), (1, 1), (1, 1)],
 [(0, 2), (0, 2), (0, 2)],
 [(0, 2), (0, 2), (1, 1)],
 [(0, 2), (1, 1), (1, 1)],
 [(1, 1), (1, 1), (1, 1)]]

In [20]:
def generate_extended_pair_partitions(inner_partitions_list, first_is_F):
    # INPUT:
    # inner_partitions_list: list of sorted tuples (each in non-increasing order) where each tuple has m_i positive
        # integers that sum to n_i, where the n_i sum to n, the total number of derivatives. 
    # first_is_F: indicates whether first sublist of inner_partitions_list corresponds to F-type fields and so 
        # does not need to be further partitioned, by contrast with bilinear fields. 
    # OUTPUT:
    # extended_pair_partitions_list: list of lists of m_i-tuples of 2-tuples, each sublist an extended pair 
    # partition, only represents Dirac bilinear fields, not F fields. 
    # EXPLANATION:
    # recursively generates a list of all sequences (lists) of pair partitions, where each pair partition is an m_i-tuple of 
    # 2-tuples, one m_i-tuple for each fielnd type. each sequence contains one pair partition for each
    # of the tuples in inner_partitions_list (one such tuple serves as an argument for generate_pair_partitions()).
    # if first inner partition in inner_partition_list corresponds to distribution of derivatives among F-type fields, omit
    # first partition. NOTE: output lists do not include numbers of derivatives acting on F-type fields. Full 
    # derivative assignment is output by generate_full_partitions(inner_partitions_list, first_is_F). 
    
    # remove any inner partition for F-type fields to focus on pair partitions of bilinear fields
    if first_is_F == True:
        inner_partitions_list = inner_partitions_list[1:]
    
    # base case
    if len(inner_partitions_list) == 1:
        partition = inner_partitions_list[0]
        pair_partitions_list = generate_pair_partitions(partition, 0)
        extended_pair_partitions_list = [[pair_partitions_list[i]] for i in range(len(pair_partitions_list))]
        return extended_pair_partitions_list
    
    # main body
    extended_pair_partitions_list = []
    first_partition = inner_partitions_list[0]
    first_pair_partitions_list = generate_pair_partitions(first_partition, 0) # list of all pair partitions of first inner partition
    extended_pair_partitions_list_old = generate_extended_pair_partitions(inner_partitions_list[1:], False)
    for i in range(len(first_pair_partitions_list)):
        extension = first_pair_partitions_list[i]
        for j in range(len(extended_pair_partitions_list_old)):
            extended_pair_partition_old = extended_pair_partitions_list_old[j]
            extended_pair_partition_new = [extension] + extended_pair_partition_old 
            extended_pair_partitions_list.append(extended_pair_partition_new)
    
    return extended_pair_partitions_list   
        

In [23]:
inner_partitions_list = [[3, 2, 2, 1], [3, 3]]
generate_extended_pair_partitions(inner_partitions_list, True)

[[[(3, 0), (3, 0)]],
 [[(3, 0), (0, 3)]],
 [[(3, 0), (2, 1)]],
 [[(3, 0), (1, 2)]],
 [[(0, 3), (0, 3)]],
 [[(0, 3), (2, 1)]],
 [[(0, 3), (1, 2)]],
 [[(2, 1), (2, 1)]],
 [[(2, 1), (1, 2)]],
 [[(1, 2), (1, 2)]]]

In [28]:
def generate_full_partitions(inner_partitions_list, first_is_F):
    # INPUT:
    # inner_partitions_list: list of sorted tuples (each in non-increasing order) where each tuple has m_i positive
        # integers that sum to n_i, where the n_i sum to n, the total number of derivatives. 
    # first_is_F: indicates whether first sublist of inner_partitions_list corresponds to F-type fields and so 
        # does not need to be further partitioned, by contrast with bilinear fields. 
    # OUTPUT:
    # full_partitions_list: list of lists of m_i-tuples. if first_is_F is True, first sublist of full_partitions_list
    # is equal to first sublist of inner_partitions_list. remaining sublists of full_partitions_list are lists of 
    # 2-tuples, each sublist a pair partition for a Dirac bilinear.
    if first_is_F:
        F_inner_partition = inner_partitions_list[0]
        full_partitions_list = []
        extended_pair_partitions_list = generate_extended_pair_partitions(inner_partitions_list, first_is_F)
        for pair_partitions_list in extended_pair_partitions_list:
            full_partition = [F_inner_partition] + pair_partitions_list
            full_partitions_list.append(full_partition)
        return full_partitions_list
    else:
        # if first sublist inner_partitions_list is not for F-type fields (because these are absent), simply
        # return the pair partitions output by generate_extended_pair_partitions()
        full_partitions_list = generate_extended_pair_partitions(inner_partitions_list, first_is_F)
        return full_partitions_list
        

In [29]:
inner_partitions_list = [[3, 2, 2, 1], [3, 3]]
generate_full_partitions(inner_partitions_list, True)

[[[3, 2, 2, 1], [(3, 0), (3, 0)]],
 [[3, 2, 2, 1], [(3, 0), (0, 3)]],
 [[3, 2, 2, 1], [(3, 0), (2, 1)]],
 [[3, 2, 2, 1], [(3, 0), (1, 2)]],
 [[3, 2, 2, 1], [(0, 3), (0, 3)]],
 [[3, 2, 2, 1], [(0, 3), (2, 1)]],
 [[3, 2, 2, 1], [(0, 3), (1, 2)]],
 [[3, 2, 2, 1], [(2, 1), (2, 1)]],
 [[3, 2, 2, 1], [(2, 1), (1, 2)]],
 [[3, 2, 2, 1], [(1, 2), (1, 2)]]]

In [17]:
def generate_inner_partitions(outer_partition, field_multiplicities):
    # INPUT:
    # outer_partition: list of numbers that sum to number of derivatives n_D with n_fields elements. 
    # field_multiplicities: list of multiplicities of fields, same length as outer_partition
    # OUTPUT:
    # subpartitions_list: list of lists of tuples, with each tuple a sorted partition of the corresponding element
    # of outer partition
    # EXPLANATION:
    # recursive function that returns a list of all distinct ways to sub-partition elements of outer partition.
    # if there are k_1 distinct ways to partition n1, k_2 ways to partition n2, ..., k_d ways to partition nd,
    # then there are k_1 x k_2 x ... x k_d distinct ways to sub-partition elements of outer partition. 

    if len(outer_partition) != len(field_multiplicities):
        print('Error: Lengths of outer_partition and field_multiplicities arguments must be equal.')
        return None

    # base case
    if len(outer_partition)==1:
        n = outer_partition[0]
        m = field_multiplicities[0]
        return [[subpartition_tuple] for subpartition_tuple in tuples_sum(m, n, order=True)]

    # main body
    subpartitions_list = [] # stores all distinct subpartitions 
    n = outer_partition[0]
    m = field_multiplicities[0]
    inner_partitions = list(tuples_sum(m, n, order=True)) 
    subpartitions = generate_inner_partitions(outer_partition[1:], field_multiplicities[1:])
    for i in range(len(inner_partitions)):
        for j in range(len(subpartitions)):
            subpartition_extended = [inner_partitions[i]] + subpartitions[j] 
            subpartitions_list.append(subpartition_extended)

    return subpartitions_list  

In [18]:
outer_partition = [1, 5, 2]
field_multiplicities = [3, 4, 3]
generate_inner_partitions(outer_partition, field_multiplicities)

[[(1, 0, 0), (5, 0, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (5, 0, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (4, 1, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 2, 0, 0), (1, 1, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (3, 1, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (2, 0, 0)],
 [(1, 0, 0), (2, 2, 1, 0), (1, 1, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (2, 0, 0)],
 [(1, 0, 0), (2, 1, 1, 1), (1, 1, 0)]]

In [24]:
def generate_derivative_assignments(field_multiset):
    # INPUT:
    # field_multiset: encodes the set of fields and derivatives making up the term
    #     e.g., field_multiset = {D:3, F:2, S:1, T:2};
    #     if a field is absent, simply omit it - e.g., write {D:3, F:2, S:1, T:2}, not {D:3, F:2, S:1, T:2, V:0}
    # OUTPUT:
    # derivative_assignments_list: A dictionary of lists of lists of tuples, where each sublist of each list is a 
    # distinct assignment of derivatives to fields.
    # EXPLANATION: generates all distinct ways of assigning derivatives to fields in field_multiset. The fields
    # S, V, T, Vp, Sp all contain two fields, a psi and a psi_bar. A derivative may be placed on either psi or psi_bar. 
    # More specifically, generate all 'outer partitions' among the field types - in the above example, F, psi_bar_S, psi_S, psi_bar_T, psi_T.
    # For each outer partition, generate all 'inner_partitions' or 'subpartitions.' Given the 
    # outer partition of n, (n1, n2, n3, n4, n5), where each of the five fields fi occurs with multiplicity mi, a 
    # subpartition is a list of tuples [(), (), (), (), ()], where the ith tuple is a sorted partition of ni with
    # mi elements. The function generates all such lists of tuples
    
    #print(field_multiset)
    # extract number of derivatives
    n_D = field_multiset['D']
    
    # find length of outer partition 
    #bilinear_keys = [k for k in list(field_multiset.keys()) if k != 'D' and k != 'F'] # keys for bilinear elements
    #bilinear_multiplicities = [field_multiset[m] for m in list(field_multiset.keys()) if m != 'D' and m != 'F']
    #n_fields = sum([1 if 'F' in field_multiset.keys() else 0]) \ # use list comprehension on list of one element 
    #    + sum([2 for key in bilinears]) # group psi and psi_bar together as one field for now
    
    #if 'F' in field_multiset.keys():
    #    F_multiplicity = [field_multiset['F']]
    #else: 
    #    F_multiplicity = []
    #bilinear_multiplicities = [x for pair in zip(bilinear_multiplicities,bilinear_multiplicities) for x in pair] 
    # for each bilinear, multiplicities are the same for psi and psi_bar
    field_multiplicities = [field_multiset[x] for x in field_multiset.keys() if x != 'D']
    
    n_fields = len(field_multiplicities)
    #print('n_fields: ' + str(n_fields))
    #print('bilinear_multiplicities: ' + str(bilinear_multiplicities))
    #print('field_multiplicities: ' + str(field_multiplicities))
    
    
    #list of lists, each sublist an outer partition, where different orderings are distinct
    outer_partitions_list = list(tuples_sum(n_fields, n_D, order=False)) 
    
    # for each outer partition, generate a list of sorted inner partitions (ordering does not matter, so one may 
    # as well sort them) where the sum of each inner partition is the corresponding element of the outer partition.
    # this is a list of lists of lists.
    #inner_partitions_dict = {}
    derivative_assignments_dict = {}
    for i in range(len(outer_partitions_list)):
        outer_partition = outer_partitions_list[i]
        #print('outer_partition: ' + str(outer_partition))
        inner_partitions_list = generate_inner_partitions(outer_partition, field_multiplicities)
        #inner_partitions_dict[outer_partition] = inner_partitions_list
        # for each outer partition and inner partition, generate a list of pair partitions
        inner_partitions_dict = {}
        for j in range(len(inner_partitions_list)):
            inner_partition = tuple(inner_partitions_list[j])
            #print('inner_partition: ' + str(inner_partition))
            full_partitions_list = generate_full_partitions(inner_partition, 'F' in field_multiset.keys())
            inner_partitions_dict[inner_partition] = full_partitions_list
        derivative_assignments_dict[outer_partition] = inner_partitions_dict

    return derivative_assignments_dict

Note that in cases where there is an $F$ field present, listed pair partitions below correspond only to elements of the corresponding inner partition after the first, since $F$ fields cannot be split into $\bar{\psi}$ and $\psi$ components. 

In [32]:
field_multiset = {'D':2, 'F': 3, 'S':3, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)


outer_partition: (2, 0, 0)
inner_partition: ((2, 0, 0), (0, 0, 0), (0, 0))
[(2, 0, 0), [(0, 0), (0, 0), (0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((1, 1, 0), (0, 0, 0), (0, 0))
[(1, 1, 0), [(0, 0), (0, 0), (0, 0)], [(0, 0), (0, 0)]]

outer_partition: (0, 2, 0)
inner_partition: ((0, 0, 0), (2, 0, 0), (0, 0))
[(0, 0, 0), [(2, 0), (0, 0), (0, 0)], [(0, 0), (0, 0)]]
[(0, 0, 0), [(0, 2), (0, 0), (0, 0)], [(0, 0), (0, 0)]]
[(0, 0, 0), [(1, 1), (0, 0), (0, 0)], [(0, 0), (0, 0)]]
inner_partition: ((0, 0, 0), (1, 1, 0), (0, 0))
[(0, 0, 0), [(1, 0), (1, 0), (0, 0)], [(0, 0), (0, 0)]]
[(0, 0, 0), [(1, 0), (0, 1), (0, 0)], [(0, 0), (0, 0)]]
[(0, 0, 0), [(0, 1), (0, 1), (0, 0)], [(0, 0), (0, 0)]]

outer_partition: (0, 0, 2)
inner_partition: ((0, 0, 0), (0, 0, 0), (2, 0))
[(0, 0, 0), [(0, 0), (0, 0), (0, 0)], [(2, 0), (0, 0)]]
[(0, 0, 0), [(0, 0), (0, 0), (0, 0)], [(0, 2), (0, 0)]]
[(0, 0, 0), [(0, 0), (0, 0), (0, 0)], [(1, 1), (0, 0)]]
inner_partition: ((0, 0, 0), (0, 0, 0), (1, 1))
[(0, 0, 0), 

In [21]:
#field_multiset = {'D':4, 'F':1, 'S':1, 'V': 2}
field_multiset = {'D':5, 'F':1, 'S':1, 'V': 2}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (5, 0, 0)
inner_partition: ((5,), (0,), (0, 0))
[[(0, 0)], [(0, 0), (0, 0)]]


outer_partition: (0, 5, 0)
inner_partition: ((0,), (5,), (0, 0))
[[(5, 0)], [(0, 0), (0, 0)]]
[[(0, 5)], [(0, 0), (0, 0)]]
[[(4, 1)], [(0, 0), (0, 0)]]
[[(1, 4)], [(0, 0), (0, 0)]]
[[(3, 2)], [(0, 0), (0, 0)]]
[[(2, 3)], [(0, 0), (0, 0)]]


outer_partition: (0, 0, 5)
inner_partition: ((0,), (0,), (5, 0))
[[(0, 0)], [(5, 0), (0, 0)]]
[[(0, 0)], [(0, 5), (0, 0)]]
[[(0, 0)], [(4, 1), (0, 0)]]
[[(0, 0)], [(1, 4), (0, 0)]]
[[(0, 0)], [(3, 2), (0, 0)]]
[[(0, 0)], [(2, 3), (0, 0)]]
inner_partition: ((0,), (0,), (4, 1))
[[(0, 0)], [(4, 0), (1, 0)]]
[[(0, 0)], [(4, 0), (0, 1)]]
[[(0, 0)], [(0, 4), (1, 0)]]
[[(0, 0)], [(0, 4), (0, 1)]]
[[(0, 0)], [(3, 1), (1, 0)]]
[[(0, 0)], [(3, 1), (0, 1)]]
[[(0, 0)], [(1, 3), (1, 0)]]
[[(0, 0)], [(1, 3), (0, 1)]]
[[(0, 0)], [(2, 2), (1, 0)]]
[[(0, 0)], [(2, 2), (0, 1)]]
inner_partition: ((0,), (0,), (3, 2))
[[(0, 0)], [(3, 0), (2, 0)]]
[[(0, 0)], [(3, 0), (0, 2)]

In case below, note that there is no $F$ field, so the number of lists in the pair partition matches the number of elements of the corresponding inner partition.

In [22]:
field_multiset = {'D':3, 'S':1, 'V': 2, 'T': 3}
derivative_assignments_dict = generate_derivative_assignments(field_multiset)

for key1 in derivative_assignments_dict.keys():
    print('')
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_dict[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_dict[key1][key2]:
            print(value)



outer_partition: (3, 0, 0)
inner_partition: ((3,), (0, 0), (0, 0, 0))
[[(3, 0)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 3)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(2, 1)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(1, 2)], [(0, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 3, 0)
inner_partition: ((0,), (3, 0), (0, 0, 0))
[[(0, 0)], [(3, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 3), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 1), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 2), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
inner_partition: ((0,), (2, 1), (0, 0, 0))
[[(0, 0)], [(2, 0), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(2, 0), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (0, 1)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
[[(0, 0)], [(1, 1), (0, 1)], [(0, 0), (0, 0), (0, 0)]]


outer_partition: (0, 0, 3)
inner_partitio

## Reducing the List of Derivative Assignments with IBP

In [23]:
from copy import deepcopy

def IBP_remove(derivative_assignment):
    # INPUT: 
    # derivative_assignment: list of lists, each sublist corresponding to a different inner partition,
    # each 2-tuple a partition of the corresponding element of the inner partition
    # OUTPUT: 
    # remove_bool: True if derivative_assignment meets requirements for removal via IBP, False otherwise
    # EXPLANATION: 
    # checks whether a derivative assignment is to be removed under the specified prescription - i.e.,
    # whether the maximum number of derivatives acting on any field is unique, and whether any field that comes 
    # earlier in the ordering of field types has exactly one fewer derivative acting on it.
    
    # first, check that maximal number of derivatives only acts on one field
    flattened_list = [x for inner_partition in derivative_assignment for pair in inner_partition for x in pair]
    print(flattened_list)
    max_derivs = max(flattened_list)
    unique_max = sum([x == max_derivs for x in flattened_list]) == 1
    print('unique_max: ' + str(unique_max ))
    
    # second, check that no field before this field has one fewer derivative acting on it
    max_index = flattened_list.index(max_derivs)
    #print('max_index: ' + str(max_index))
    if max_index > 0:
        one_less_before = max(flattened_list[:max_index]) == max_derivs - 1
    else:
        one_less_before = False
    print('one_less_before: ' + str(one_less_before))
    
    if unique_max and not one_less_before:
        return True
    
    return False


def IBP_reduce(derivative_assignments_dict):
    # INPUT:
    # derivative_assignments_dict: dictionary containing all possible derivative assignments for a given
    # field/derivative multiset. 
    # OUTPUT:
    # reduced_derivative_assignments_dict: a dictionary containing an IBP-reduced set of derivative assignments
    # removed_derivative_assignments_dict: a dictionary containing all removed derivative assignments
    # EXPLANATION:
    # scans through all derivative assignments and performs IBP reduction. it removes one operator for each 
    # IBP relation, which appears in one and only one IBP relation. in practice, it does this by removing 
    # any derivative assignments for which the maximum number of derivatives acting on any field is unique, and
    # for which there is not one fewer derivative acting on any field that occurs earlier in the specified ordering
    # of field types. it is worth noting that there is an element of conventionality to this prescription, and that 
    # other IBP reductions are possible that remove different sets of derivative assignments. 
    
    #derivative_assignments_dict_IBP_reduced
    deleted = []
    derivative_assignments_dict_reduced = deepcopy(derivative_assignments_dict)
    
    for outer_partition in derivative_assignments_dict.keys():
        #print('outer partition: ' + str(outer_partition))
        inner_partitions_dict = derivative_assignments_dict[outer_partition] 
        for inner_partition in inner_partitions_dict.keys():
            #print('inner partition: ' + str(inner_partition))
            extended_pair_partitions_list = inner_partitions_dict[inner_partition]
            extended_pair_partitions_list_reduced = derivative_assignments_dict_reduced[outer_partition][inner_partition]
            for derivative_assignment in extended_pair_partitions_list:
                print(derivative_assignment)
                if IBP_remove(derivative_assignment):
                    #print('REMOVE: ' + str(derivative_assignment))
                    deleted.append(derivative_assignment)
                    #print(len(derivative_assignments_dict[outer_partition][inner_partition]))
                    extended_pair_partitions_list_reduced.remove(derivative_assignment) 
                    #print(len(derivative_assignments_dict[outer_partition][inner_partition]))
        
    return derivative_assignments_dict_reduced, deleted





In [24]:
derivative_assignment = [[(0, 0)], [(1, 0), (0, 0)]]
IBP_remove(derivative_assignment)

[0, 0, 1, 0, 0, 0]
unique_max: True
one_less_before: True


False

In [25]:
field_multiset = {'D':2, 'F':1, 'S':1, 'V': 2}
derivative_assignments = generate_derivative_assignments(field_multiset)
derivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)

for key1 in derivative_assignments_IBP_reduced.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_IBP_reduced[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_IBP_reduced[key1][key2]:
            print(value)


[[(0, 0)], [(0, 0), (0, 0)]]
[0, 0, 0, 0, 0, 0]
unique_max: False
one_less_before: False
[[(2, 0)], [(0, 0), (0, 0)]]
[2, 0, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(0, 2)], [(0, 0), (0, 0)]]
[0, 2, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(1, 1)], [(0, 0), (0, 0)]]
[1, 1, 0, 0, 0, 0]
unique_max: False
one_less_before: False
[[(0, 0)], [(2, 0), (0, 0)]]
[0, 0, 2, 0, 0, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(0, 2), (0, 0)]]
[0, 0, 0, 2, 0, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(1, 1), (0, 0)]]
[0, 0, 1, 1, 0, 0]
unique_max: False
one_less_before: True
[[(0, 0)], [(1, 0), (1, 0)]]
[0, 0, 1, 0, 1, 0]
unique_max: False
one_less_before: True
[[(0, 0)], [(1, 0), (0, 1)]]
[0, 0, 1, 0, 0, 1]
unique_max: False
one_less_before: True
[[(0, 0)], [(0, 1), (0, 1)]]
[0, 0, 0, 1, 0, 1]
unique_max: False
one_less_before: True
[[(1, 0)], [(0, 0), (0, 0)]]
[1, 0, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(0, 1)], [(0, 0), (0, 0)]]
[

In [26]:
IBP_remove([[(0, 3)], [(0, 0), (0, 0)]])

[0, 3, 0, 0, 0, 0]
unique_max: True
one_less_before: False


True

In [27]:
#derivative_assignment = [[(0, 0)], [(2, 0), (1, 0)], [(0, 0), (0, 0), (0, 0)]]
#derivative_assignment = [[(1, 2)], [(0, 0), (0, 0)], [(0, 1), (0, 0), (0, 0)]]
#derivative_assignment = [[(3, 2)], [(0, 0), (0, 0)]]
#derivative_assignment = [[(0, 0)], [(1, 4), (0, 0)]]
#derivative_assignment = [[(1, 0)], [(2, 0), (0, 0)], [(0, 0), (0, 0), (0, 0)]]
#derivative_assignment = [[(0, 0)], [(2, 2), (1, 0)]]
#derivative_assignment = [[(0, 0)], [(2, 1), (2, 0)]]
#derivative_assignment = [[(0, 0)], [(1, 3), (1, 0)]]
derivative_assignment = [[(0, 3)], [(0, 0), (0, 0)]]
IBP_remove(derivative_assignment)

[0, 3, 0, 0, 0, 0]
unique_max: True
one_less_before: False


True

In [28]:
for item in deleted:
    print(item)

[[(2, 0)], [(0, 0), (0, 0)]]
[[(0, 2)], [(0, 0), (0, 0)]]
[[(0, 0)], [(2, 0), (0, 0)]]
[[(0, 0)], [(0, 2), (0, 0)]]
[[(1, 0)], [(0, 0), (0, 0)]]


In [29]:
# compare derivative_assignments and derivative_assignments_IBP_reduced
for outer_partition in derivative_assignments.keys():
    for inner_partition in derivative_assignments[outer_partition].keys():
        print(len(derivative_assignments[outer_partition][inner_partition]))
print('')
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        print(len(derivative_assignments_IBP_reduced[outer_partition][inner_partition]))

print('')
num_removed = 0
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        num_removed += len(derivative_assignments[outer_partition][inner_partition]) - len(derivative_assignments_IBP_reduced[outer_partition][inner_partition])
print(num_removed)

        

1
3
3
3
2
2
4

1
1
1
3
1
2
4

5


In [30]:
#field_multiset = {'D':2, 'F':1, 'S':1, 'V': 1}
field_multiset = {'D':3, 'F':1, 'S':1, 'V': 2}
derivative_assignments = generate_derivative_assignments(field_multiset)
print(derivative_assignments)


{(3, 0, 0): {((3,), (0,), (0, 0)): [[[(0, 0)], [(0, 0), (0, 0)]]]}, (0, 3, 0): {((0,), (3,), (0, 0)): [[[(3, 0)], [(0, 0), (0, 0)]], [[(0, 3)], [(0, 0), (0, 0)]], [[(2, 1)], [(0, 0), (0, 0)]], [[(1, 2)], [(0, 0), (0, 0)]]]}, (0, 0, 3): {((0,), (0,), (3, 0)): [[[(0, 0)], [(3, 0), (0, 0)]], [[(0, 0)], [(0, 3), (0, 0)]], [[(0, 0)], [(2, 1), (0, 0)]], [[(0, 0)], [(1, 2), (0, 0)]]], ((0,), (0,), (2, 1)): [[[(0, 0)], [(2, 0), (1, 0)]], [[(0, 0)], [(2, 0), (0, 1)]], [[(0, 0)], [(0, 2), (1, 0)]], [[(0, 0)], [(0, 2), (0, 1)]], [[(0, 0)], [(1, 1), (1, 0)]], [[(0, 0)], [(1, 1), (0, 1)]]]}, (2, 1, 0): {((2,), (1,), (0, 0)): [[[(1, 0)], [(0, 0), (0, 0)]], [[(0, 1)], [(0, 0), (0, 0)]]]}, (2, 0, 1): {((2,), (0,), (1, 0)): [[[(0, 0)], [(1, 0), (0, 0)]], [[(0, 0)], [(0, 1), (0, 0)]]]}, (1, 2, 0): {((1,), (2,), (0, 0)): [[[(2, 0)], [(0, 0), (0, 0)]], [[(0, 2)], [(0, 0), (0, 0)]], [[(1, 1)], [(0, 0), (0, 0)]]]}, (1, 0, 2): {((1,), (0,), (2, 0)): [[[(0, 0)], [(2, 0), (0, 0)]], [[(0, 0)], [(0, 2), (0, 0)]]

In [31]:
# compare derivative_assignments and derivative_assignments_IBP_reduced
for outer_partition in derivative_assignments.keys():
    for inner_partition in derivative_assignments[outer_partition].keys():
        print(len(derivative_assignments[outer_partition][inner_partition]))

1
4
4
6
2
2
3
3
3
6
6
6
4


In [32]:
derivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)
len(deleted)

[[(0, 0)], [(0, 0), (0, 0)]]
[0, 0, 0, 0, 0, 0]
unique_max: False
one_less_before: False
[[(3, 0)], [(0, 0), (0, 0)]]
[3, 0, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(0, 3)], [(0, 0), (0, 0)]]
[0, 3, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(2, 1)], [(0, 0), (0, 0)]]
[2, 1, 0, 0, 0, 0]
unique_max: True
one_less_before: False
[[(1, 2)], [(0, 0), (0, 0)]]
[1, 2, 0, 0, 0, 0]
unique_max: True
one_less_before: True
[[(0, 0)], [(3, 0), (0, 0)]]
[0, 0, 3, 0, 0, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(0, 3), (0, 0)]]
[0, 0, 0, 3, 0, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(2, 1), (0, 0)]]
[0, 0, 2, 1, 0, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(1, 2), (0, 0)]]
[0, 0, 1, 2, 0, 0]
unique_max: True
one_less_before: True
[[(0, 0)], [(2, 0), (1, 0)]]
[0, 0, 2, 0, 1, 0]
unique_max: True
one_less_before: False
[[(0, 0)], [(2, 0), (0, 1)]]
[0, 0, 2, 0, 0, 1]
unique_max: True
one_less_before: False
[[(0, 0)], [(0, 2), (1, 0)]]
[0, 

19

In [33]:
derivative_assignments_IBP_reduced

{(3, 0, 0): {((3,), (0,), (0, 0)): [[[(0, 0)], [(0, 0), (0, 0)]]]},
 (0, 3, 0): {((0,), (3,), (0, 0)): [[[(1, 2)], [(0, 0), (0, 0)]]]},
 (0, 0, 3): {((0,), (0,), (3, 0)): [[[(0, 0)], [(1, 2), (0, 0)]]],
  ((0,), (0,), (2, 1)): [[[(0, 0)], [(1, 1), (1, 0)]],
   [[(0, 0)], [(1, 1), (0, 1)]]]},
 (2, 1, 0): {((2,), (1,), (0, 0)): [[[(0, 1)], [(0, 0), (0, 0)]]]},
 (2,
  0,
  1): {((2,), (0,), (1, 0)): [[[(0, 0)], [(1, 0), (0, 0)]],
   [[(0, 0)], [(0, 1), (0, 0)]]]},
 (1, 2, 0): {((1,), (2,), (0, 0)): [[[(1, 1)], [(0, 0), (0, 0)]]]},
 (1, 0, 2): {((1,), (0,), (2, 0)): [[[(0, 0)], [(1, 1), (0, 0)]]],
  ((1,), (0,), (1, 1)): [[[(0, 0)], [(1, 0), (1, 0)]],
   [[(0, 0)], [(1, 0), (0, 1)]],
   [[(0, 0)], [(0, 1), (0, 1)]]]},
 (0,
  2,
  1): {((0,), (2,), (1, 0)): [[[(1, 1)], [(1, 0), (0, 0)]],
   [[(1, 1)], [(0, 1), (0, 0)]]]},
 (0,
  1,
  2): {((0,), (1,), (2, 0)): [[[(1, 0)], [(2, 0), (0, 0)]],
   [[(1, 0)], [(0, 2), (0, 0)]],
   [[(1, 0)], [(1, 1), (0, 0)]],
   [[(0, 1)], [(2, 0), (0, 0)]],
  

In [34]:
for outer_partition in derivative_assignments_IBP_reduced.keys():
    for inner_partition in derivative_assignments_IBP_reduced[outer_partition].keys():
        print(len(derivative_assignments_IBP_reduced[outer_partition][inner_partition]))

1
1
1
2
1
2
1
1
3
2
6
6
4


## Generate All Contractions of Free Indices in Multiset with Derivative Assignments

To avoid generation of equivalent Lorentz contractions, it is useful to group objects with interchangeable indices, or sets of fields with interchangeable sets of indices, together. For example, consider the multiset $(DDD F) V$. If one takes a naive approach and numbers the five fields $0, 1, 2, 3, 4, 5$ and then generates all possible contractions of these five objects, the process will generate many repeats. For example, the set of contractions $[(0,1), (2,3), ...]$ gives the same term as $[(0,3), (1,2), ...]$ because of the interchangeability of derivatives. Also, in the multiset $(DF) (DF) V V$, the set of contractions $[(0,5), (1,2), (1,3), (3,4)]$ gives the same result as if we interchange the indices $0,1$ associated with the first $DF$ with the indices $2,3$ associated with the second $DF$, so that $0 <-> 2$ and $1 <-> 3$ - so, for example, $[(2,5), (3,0), (3,1), (1,4)]$. 

To avoid the inefficiency of generating equivalent ways of Lorentz contracting fields, and the complication of having to remove repeated terms, it is prudent first to group field indices, or sets of field indices, that are mutually interchangeable. When encoding a particular way of contracting Lorentz indices in a given multiset and derivative assignment, the contractions are encoded in terms of these groups rather than in terms of individual fields or indices. This includes all derivatives acting on the same field. It also includes multiple copies of the same field type with the same number of derivatives acting on them. Thus, the strategy is first to encode the grouping of indices associated with a given multiset and derivative assignment, and then to systematically generate inequivalent ways of contracting Lorentz indices. 

In [218]:
'''The following cell is modified from an older version of this code in the folder QED/New_Approach'''
'''
from multiset import FrozenMultiset

def lorentzRanks_list(field_multiset, lorentzRanks_dict):
    symbol_list = list(field_multiset.keys())
    numFields_list = list(field_multiset.values())
    lorentzRanks_list = [lorentzRanks_dict[key] for key in field_multiset.keys()]
    N_fieldTypes = len(symbol_list)
    lorentzRanks = [num_fields*lor_rank for num_fields,lor_rank  in zip(numFields_list, lorentzRanks_list)]
    return lorentzRanks

  
def generate_pre_lorentz_contractions(lorentzRanks_list, i0):
    # EXPLANATION: recursively generates all ways of lorentz contracting fields. generate a set of sets of contractions. all sets of
    # contractions should have the same number of contractions. take first index in lorentzRanks_list and contract it with every field 
    # with non-zero lorentz rank. 
    # for each such contraction, update the lorentzRanks_list by decrementing the lorentz rank of each contracted
    # field by one for each contracted index. append this contraction to each of the sets of contractions output 
    # by generate_pre_lorentz_contractions() acting on the decremented lorentzRanks_list. 
    
    # base case: if the total number of free Lorentz indices is 0 or 1, return empty list of contraction sets 
    if sum(lorentzRanks_list) < 2:
        contraction_list_list = [[]]
        return contraction_list_list
    
    # if sum of lorentz ranks is odd, there must be one uncontracted index remaining after all contractions. 
    # this free index can be associated with any field. for each field with non-zero lorentz rank, generate all 
    # contractions of fields where free index is attached to that field. do this by decrementing field rank by one
    # and then generating contractions for lorentz rank list with even total lorentz rank. 
    if sum(lorentzRanks_list) %2 != 0:
        N_fieldTypes = len(lorentzRanks_list)
        contraction_list_list = []
        for i in range(N_fieldTypes):
            if lorentzRanks_list[i] > 0:
                lorentzRanks_list_new = lorentzRanks_list.copy()
                lorentzRanks_list_new[i] -= 1
                contraction_list_list += generate_pre_lorentz_contractions(lorentzRanks_list_new, 0)
        return contraction_list_list
                
    # otherwise, if sum of lorentzRanks_list is even, proceed as follows.
    
    # set to store different sets of contractions
    contraction_list_list = []
    # variable to store number of distinct fields
    N_fieldTypes = len(lorentzRanks_list)
    
    # find first field of non-zero rank for first index of contraction
    for i in range(N_fieldTypes):
        if lorentzRanks_list[i] >= 1:
            i_start = i
            lorentzRanks_list[i_start] -= 1
            #print('i_start: ' + str(i_start))
            break
            
    if i0 <= i_start:
        i0 = i_start
     
    # find all ways of contracting first non-zero lorentz rank field with other fields or itself.
    for i in range(i0, N_fieldTypes):
        #print(i)
        if lorentzRanks_list[i] >= 1:
            contraction = (i_start, i)
            
            lorentzRanks_list_old = lorentzRanks_list.copy()
            
            #decrement lorentzRanks_list_old 
            lorentzRanks_list_old[i] -= 1
            
            #if i_start of decremented list is the same as the non-decremented list, only include contractions
            #where second index is greater than or equal to i
            if lorentzRanks_list_old[i_start] > 0:
                contraction_list_list_old = generate_pre_lorentz_contractions(lorentzRanks_list_old, i)
            else: 
                contraction_list_list_old = generate_pre_lorentz_contractions(lorentzRanks_list_old, 0)
            
            for contraction_list_old in contraction_list_list_old:
                contraction_list = contraction_list_old.copy()
                contraction_list.append(contraction)
                contraction_list_list.append(contraction_list) 
                
    return contraction_list_list


def generate_pre_lorentz_contractions_from_dict(field_multiset, lorentzRanks_dict):
    lorentzRanks = lorentzRanks_list(field_multiset, lorentzRanks_dict)
    contraction_list_list = generate_pre_lorentz_contractions(lorentzRanks, 0)
    contraction_multisets_set = set()
    for item in contraction_list_list:
        contraction_multiset = FrozenMultiset(item)
        contraction_multisets_set.add(contraction_multiset)
    return contraction_multisets_set
'''

In [224]:
'''
field_multiset = {'D':3, 'F':1, 'S':1, 'V': 3}
lorentzRanks_dict = {'D':1, 'F':2, 'S':0, 'V':1, 'T':2, 'Vp':1, 'Sp':0}
#lorentzRanks_list(field_multiset, lorentzRanks_dict)
pre_lorentz_contractions = generate_pre_lorentz_contractions_from_dict(field_multiset, lorentzRanks_dict)
print(pre_lorentz_contractions)
    
print('')
derivative_assignments = generate_derivative_assignments(field_multiset)
derivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)
for key1 in derivative_assignments_IBP_reduced.keys():
    print('')
    print('outer_partition: ' + str(key1))
    for key2 in derivative_assignments_IBP_reduced[key1].keys():
        print('inner_partition: ' + str(key2))
        for value in derivative_assignments_IBP_reduced[key1][key2]:
            print(value)
'''

"\nfield_multiset = {'D':3, 'F':1, 'S':1, 'V': 3}\nlorentzRanks_dict = {'D':1, 'F':2, 'S':0, 'V':1, 'T':2, 'Vp':1, 'Sp':0}\n#lorentzRanks_list(field_multiset, lorentzRanks_dict)\npre_lorentz_contractions = generate_pre_lorentz_contractions_from_dict(field_multiset, lorentzRanks_dict)\nprint(pre_lorentz_contractions)\n    \nprint('')\nderivative_assignments = generate_derivative_assignments(field_multiset)\nderivative_assignments_IBP_reduced, deleted = IBP_reduce(derivative_assignments)\nfor key1 in derivative_assignments_IBP_reduced.keys():\n    print('')\n    print('outer_partition: ' + str(key1))\n    for key2 in derivative_assignments_IBP_reduced[key1].keys():\n        print('inner_partition: ' + str(key2))\n        for value in derivative_assignments_IBP_reduced[key1][key2]:\n            print(value)\n"

Having generated all pre Lorentz contractions for a multiset without derivative assignments, let us know generate all Lorentz contractions consistent with a particular derivative assignment and pre Lorentz contraction.

In [ ]:
def generate_lorentz_contractions(field_multiset, contraction_list, derivative_assignment):
    
    return lorentz_contraction_lists_list

In [ ]:
field_multiset = {'D':5, 'F':1, 'S':1, 'V': 4}
# [(D)(F)][(S)][(DD)Pb (_V_) (D)P][(D)Pb (_V_) P][(V)][(V)]
'''
derivative_assignment = 
outer_partition: (1, 0, 2)
inner_partition: ((1,), (0,), (3, 1, 0))
[[(0, 0)], [(2, 1), (1, 0), (0, 0)]]
'''